In [1]:
# !pip install torch torchvision tqdm matplotlib onnx onnxscript
# sudo docker run -it --rm --gpus all --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 -v$(pwd):/run/host -p 8888:8888 nvcr.io/nvidia/pytorch:24.12-py3


import random
import os

import torch
torch.set_float32_matmul_precision('high')
import torch.utils.data as tud
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as tvt
import torchvision.transforms.v2 as tv2
import torchvision.transforms.functional as tvf
import torchvision.datasets as tds
import torchvision.utils as tu
import torchvision

from tqdm import tqdm
import matplotlib.pyplot as plt

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
cpu_num = os.cpu_count() // 2

In [2]:
import loaders
batchsize = 64
DIM, train_loader, val_loader = loaders.get_loaders(batchsize)

In [3]:
model = torchvision.models.resnet18(weights=None)
# Replace the head b/c imagenette has only 10 classes.
model.fc = nn.Linear(in_features=512, out_features=10, bias=True)

model = model.to(device).train()

In [4]:
from apex.contrib.sparsity import ASP

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
epochs = 30
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

ASP.prune_trained_model(model, optimizer)

lossfn = nn.CrossEntropyLoss()
loss_plot = []
for epoch in range(epochs):
    model.train()
    for i, (images, target) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        images = images.to(device)
        targets = target.to(device)

        outs = model(images)
        loss = lossfn(outs, targets)
        loss.backward()
        optimizer.step()
    scheduler.step()

    losses = []
    model.eval()
    correct = 0
    total = len(val_loader.dataset)
    for i, (images, target) in enumerate(val_loader):
        with torch.no_grad():
            images = images.to(device)
            targets = target.to(device)
            outs = model(images)

            loss = lossfn(outs, targets)
            losses.append(loss)
            for x in range(outs.shape[0]):
                preds = F.softmax(outs, dim=1)
                cls = preds[x].argmax()
                lbl = targets[x]
                if cls == lbl:
                    correct += 1

    epoch_loss = torch.Tensor(losses).mean().item()
    print("Epoch {}: {} ".format(epoch, epoch_loss))
    print("Current LR is {}".format(scheduler.get_last_lr()))
    print("{}/{} correct, {:.2f}%".format(correct, total, 100*correct/total))
    loss_plot.append(epoch_loss)

Found permutation search CUDA kernels
[ASP][Info] permutation_search_kernels can be imported.
[ASP] torchvision is imported, can work with the MaskRCNN/KeypointRCNN from torchvision.
[ASP] Auto skipping pruning conv1::weight of size=torch.Size([64, 3, 7, 7]) and type=torch.float32 for sparsity
[ASP] Auto skipping pruning fc::weight of size=torch.Size([10, 512]) and type=torch.float32 for sparsity
[set_permutation_params_from_asp] Set permutation needed parameters
	Sparse parameter names: ['layer1.0.conv1:weight', 'layer1.0.conv2:weight', 'layer1.1.conv1:weight', 'layer1.1.conv2:weight', 'layer2.0.conv1:weight', 'layer2.0.conv2:weight', 'layer2.0.downsample.0:weight', 'layer2.1.conv1:weight', 'layer2.1.conv2:weight', 'layer3.0.conv1:weight', 'layer3.0.conv2:weight', 'layer3.0.downsample.0:weight', 'layer3.1.conv1:weight', 'layer3.1.conv2:weight', 'layer4.0.conv1:weight', 'layer4.0.conv2:weight', 'layer4.0.downsample.0:weight', 'layer4.1.conv1:weight', 'layer4.1.conv2:weight']
	All param

/usr/local/lib/python3.12/dist-packages/apex/contrib/sparsity/sparse_masklib.py:42: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:78.)
  mask = torch.cuda.IntTensor(matrix.shape).fill_(1).view(-1,m)


[accelerated_search_for_good_permutation] Take 2.8250 seconds to search the permutation sequence.
[search_for_good_permutation] Take 2.8318 seconds to finish accelerated_search_for_good_permutation function and with final magnitude 29342.548828125.
Permutation for sibling group 12: [59, 130, 312, 38, 4, 275, 326, 511, 8, 142, 170, 454, 12, 105, 319, 90, 205, 300, 104, 284, 20, 349, 320, 198, 24, 224, 147, 145, 178, 301, 240, 161, 428, 210, 47, 385, 252, 88, 444, 487, 291, 61, 318, 383, 190, 256, 504, 106, 48, 427, 49, 437, 52, 403, 35, 507, 328, 78, 330, 331, 60, 134, 11, 414, 453, 64, 389, 321, 68, 71, 69, 276, 72, 486, 119, 112, 334, 100, 333, 509, 431, 265, 442, 117, 44, 348, 314, 128, 66, 323, 239, 322, 203, 168, 163, 13, 41, 257, 466, 136, 42, 196, 40, 98, 471, 367, 227, 407, 108, 350, 338, 441, 484, 46, 343, 340, 116, 299, 264, 366, 295, 120, 292, 353, 267, 255, 364, 365, 74, 186, 129, 75, 132, 496, 109, 285, 317, 200, 97, 73, 191, 409, 499, 341, 110, 344, 131, 144, 148, 149, 324


/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:217: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 0: 2.5715417861938477 
Current LR is [0.009972609476841367]
1061/3925 correct, 27.03%


...

Epoch 27: 0.5127426385879517 
Current LR is [0.00010926199633097157]
3306/3925 correct, 84.23%



00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 148/148 [00:02<00:00, 53.95it/s]

Epoch 28: 0.506304919719696 
Current LR is [2.7390523158632995e-05]
3304/3925 correct, 84.18%



00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 148/148 [00:02<00:00, 53.84it/s]

Epoch 29: 0.5037730932235718 
Current LR is [0.0]
3311/3925 correct, 84.36%


In [5]:
torch.save(model, "sparse_resnet.pth")

In [6]:
# TODO Tensorrt has problems importing the model if it's exported by dynamo.
torch_input = torch.randn(1, 3, DIM, DIM).to(device)
onnx_program = torch.onnx.export(
    model,
    (torch_input,),
    "sparse_resnet.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamo=False,
    external_data=False,
)

In [7]:
from ipywidgets import interact

@interact(index=(0, len(val_loader.dataset) - 1, 1))
def draw_preds(index=0):
    model.eval()
    with torch.no_grad():
        image = val_loader.dataset[index][0]
        pred = model(image.float().unsqueeze(0).to(device))
        pred = F.softmax(pred, dim=1)
        clsid = pred.argmax()
        plt.imshow(image.float().cpu().squeeze().permute(1, 2, 0), cmap='gray')
        print(int(clsid))

interactive(children=(IntSlider(value=0, description='index', max=3924), Output()), _dom_classes=('widget-inte…